# An Explain Report

This is a POC for an explain report: given a number at time T1 and time T2, introspect the clocks involved to produce a simple linear breakdown
of what changed on various timelines to account for the change in the number between T1 and T2.

The report is an ordered sequence of clock changes: the sum of changes should equal to the total change. The order of application is important: change the
ordering, and the resultant values may change. It's not a Jacobian-type report with T1 deltas and no ordering.

The real reason for this workbook is to start working on the node class optimizations and metadata management.

## Footnotes

Footnotes are one of the simplest types of metadata:
* Any computation (even a simple read of a constant value) may declare it has footnotes
* Every computation gets the footnote of all its inputs

Footnoting is based on a number of observations about large software systems (say, 1M LOC or more):
* Any complex report (i.e. computation output) will generally be somewhat wrong, misleading, or out of date.
* If you want 100% correctness and truth, your report will raise an exception every time you run it.
* If you just want it to run, you'll get a result, but some log file on some machine in some compute farm will have a message explaining why your result is wrong.
* And you won't see that message.
* And that message may have leaked information that bad actors can read.
* And you won't trust this system, so your own developers will copy the underlying data and write their own version of the report with all the same issues.
* Now you don't have a problem, but your company now has two problems.

Footnotes summary:
* Are the documentation of problems from the producer's point of view
* Make no claim about usability of results, etc, for a specific consumer
* May be programmatically removed/condensed/replaced at controlled code points


In [1]:
from mand.core import Entity, node, Context
from mand.core import ObjectDb, Timestamp, getNode, _tr
from mand.core import ProfileMonitor
from mand.core import displayDict, displayMarkdown, displayListOfDicts, displayHeader
from mand.core import find
from mand.demos.trading import makeWorld, bookSomeTrades

db = ObjectDb()

from mand.lib.dbsetup import setUpDb
setUpDb(db)
db.describe()

ImportError: No module named mand.core

In [ ]:
with db:
    pWorld = makeWorld()
    
pAll, bExt, bExt2 = pWorld.children()
p1 = pAll.children()[0]
p2 = pAll.children()[1]
p4 = pAll.children()[3]

b2 = p2.children()[0]
b4 = p4.children()[0]

# Book some trades

This time, we throw a few thousand in...

In [ ]:
ts0, ts1, ts2, ts3, ts4, ts5, eod, ts6 = bookSomeTrades(pWorld)

# An Explain Report

This is a mix of abstractions at the Core, DBA, and BA, and User levels. But, it does the job for now...

In [ ]:
class Report(Entity):
    @node(stored=True)
    def valuable(self):
        return None
    
    @node(stored=True)
    def ts1(self):
        return None
    
    @node(stored=True)
    def ts2(self):
        return None
    
    @node
    def cutoffs(self):
        valuable = self.valuable()
        ts1 = self.ts1()
        ts2 = self.ts2()
        
        clock = valuable.getObj(_tr.RootClock, 'Main')
        
        def cuts(ts):
            def fn(node):
                obj = node.key.object()
                m = node.key.shortName()
                if isinstance(obj, _tr.Clock) and m == 'cutoffs':
                    return True
            with Context({clock.cutoffs: ts}, 'Clocks'):
                nodes = find(valuable.NPV, fn)
                return nodes

        ret = {}
        for n in cuts(ts1):
            ret[n.key._key] = n
        for n in cuts(ts2):
            ret[n.key._key] = n
        return ret.values()
    
    @node
    def data(self):
        valuable = self.valuable()
        ts1 = self.ts1()
        ts2 = self.ts2()
        clock = valuable.getObj(_tr.RootClock, 'Main')
    
        nodes = self.cutoffs()
    
        # IRL, we'd sort these according to some business req...
        # And our clocks might be arranged in an N-level tree...
        nodes = sorted(nodes, key = lambda node: node.object().meta.name())
    
        data = []
        curr = [0]
        def add(title, npv):
            pnl = npv - curr[0]
            curr[0] = npv
            data.append( {'Activity': title, 'PnL': pnl } )

        with Context({clock.cutoffs: ts1}, 'Start'):
            curr = [ valuable.NPV() ] # Starting balance
    
        tweaks = {}
        for n in nodes:
            tweaks[n] = ts1
        with Context(tweaks, name='Start breaks'):
            start = valuable.NPV()
            add('Starting balance breaks', start)

        tsAmend = Timestamp(t=ts2.transactionTime, v=ts1.validTime)
        # XXX - modifying tweaks in place is a bit evil
        # This is only safe because I know Context() effectively copies, so this works
        # for now.
        for n in nodes:
            tweaks[n] = tsAmend
            name = n.object().meta.name()
            with Context(tweaks, name='Amend %s' % name):
                add('prior day amends: %s' % name, valuable.NPV())
        for n in nodes:
            tweaks[n] = ts2
            name = n.object().meta.name()
            with Context(tweaks, name='Activity %s' % name):
                add('activity: %s' % name, valuable.NPV())
    
        with Context({clock.cutoffs: ts2}, name='End'):
            end = valuable.NPV()
            add('Ending balance breaks', end)

        title = 'PnL explain for %s: %s' % (valuable.meta.name(), end-start)
        return data, title

    def run(self):
        data, title = self.data()
        node = getNode(self.data)
        footnotes = node.footnotes.values()
        displayHeader('%s' % title)
        if footnotes:
            displayMarkdown('**Caveat: this report encountered problems. See footnotes at bottom.**')
        displayListOfDicts(data, names=['Activity', 'PnL'] )
        if footnotes:
            displayMarkdown('## Footnotes:')
            displayMarkdown(node.footnoteMarkdown())
    
r = Report(valuable=pAll, ts1=eod, ts2=ts6)
r.run()

## Add some inconsistent data [Test]

Book b2 should appear multiple times in some portfolio trees and be flagged accordingly...

In [ ]:
with db:
    p1.setChildren(p1.children() + [b2])

ts7 = Timestamp()

# Footnotes

Note the report calculation has run, but attached appropriate caveats to the output:

In [ ]:
db3 = db.copy()
p = db3._get(pAll.meta.path())

with ProfileMonitor(mode='sum'): 
    r = Report(valuable=p, ts1=eod, ts2=ts7)
    r.run()